In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
import pickle

def load_processed_data(file_path):
    with open(file_path, 'rb') as file:
        X, Y = pickle.load(file)
    return X, Y

In [5]:
X, Y = load_processed_data('./pickel_files/no_buffer_res_all_data.pkl')

In [6]:

X_filtered = [img for img in X if img is not None and isinstance(img, np.ndarray)]
Y_filtered = [Y[i] for i in range(len(Y)) if X[i] is not None and isinstance(X[i], np.ndarray)]

X_filtered = np.array(X_filtered)

Y_filtered = np.array(Y_filtered)
Y_filtered = Y_filtered[:, :2]

In [7]:
len(X_filtered), len(Y_filtered)

(3978, 3978)

In [8]:
Y_filtered.shape

(3978, 2)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X_filtered, Y_filtered, test_size=0.2, random_state=42)
#Val data
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

train_datagen = ImageDataGenerator(
    # rotation_range=5,
    # width_shift_range=0.02,
    # height_shift_range=0.02,
    # zoom_range=[0.95, 1.05],
    # brightness_range=[0.8, 1.2],
    # shear_range=0.1,
    # fill_mode='nearest'
)


# Assuming you have your training data in train_data and train_labels
train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)


model = Sequential()

# Adding L2 Regularization to Convolutional Layers
l2_reg = 0.001

# First Conv Block
model.add(Conv2D(32, (7, 7), activation='relu', input_shape=(48, 80, 3), kernel_regularizer=l2(l2_reg)))

# Second Conv Block
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(Dropout(0.15))

model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(Dropout(0.15))

# Third Conv Block
model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D())
model.add(BatchNormalization())


# Fourth Conv Block
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))

# Fifth Conv Block
model.add(Conv2D(1028, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))

# Sixth Conv Block
model.add(Conv2D(2056, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))


# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00005), loss='mse', metrics=['mean_squared_error', 'mean_absolute_error'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=100,  # Adjust number of epochs
    validation_data=(X_val, Y_val),  # Assuming validation data is available
      callbacks=[early_stopping],
    batch_size=16
)

Epoch 1/100
80/80 [==============================] - 19s 107ms/step - loss: 0.2431 - mean_squared_error: 0.0744 - mean_absolute_error: 0.2189 - val_loss: 0.2648 - val_mean_squared_error: 0.1043 - val_mean_absolute_error: 0.2808
Epoch 2/100
80/80 [==============================] - 6s 72ms/step - loss: 0.2057 - mean_squared_error: 0.0529 - mean_absolute_error: 0.1783 - val_loss: 0.2457 - val_mean_squared_error: 0.1005 - val_mean_absolute_error: 0.2753
Epoch 3/100
80/80 [==============================] - 6s 72ms/step - loss: 0.1833 - mean_squared_error: 0.0450 - mean_absolute_error: 0.1639 - val_loss: 0.2253 - val_mean_squared_error: 0.0938 - val_mean_absolute_error: 0.2661
Epoch 4/100
80/80 [==============================] - 6s 73ms/step - loss: 0.1640 - mean_squared_error: 0.0387 - mean_absolute_error: 0.1497 - val_loss: 0.2045 - val_mean_squared_error: 0.0852 - val_mean_absolute_error: 0.2525
Epoch 5/100
80/80 [==============================] - 6s 75ms/step - loss: 0.1491 - mean_square

KeyboardInterrupt: 

In [10]:
#evaluate the model
model.evaluate(X_test, Y_test)

NameError: name 'model' is not defined

In [12]:
from keras.models import load_model
model = load_model('./models/eye_gaze_v17.h5')

In [13]:
#plot predicted vs actual on test data on a canvas using opencv 

import cv2
import numpy as np 
predictions = model.predict(X_test)
screen_width, screen_height = 2650, 1440
canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
cv2.namedWindow('Gaze Tracking on Canvas', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Gaze Tracking on Canvas', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# plot the first 10 images one by one
for i in range(0,25):
    
    # get the predicted x,y coordinates
    x, y = predictions[i][0] * screen_width, predictions[i][1] * screen_height

    # lock the preds 
    x = min(max(x, 0), screen_width)
    y = min(max(y, 0), screen_height)

    # get the actual x,y coordinates
    x_actual, y_actual = Y_test[i][0] * screen_width, Y_test[i][1] * screen_height

    # plot the predicted x,y coordinates
    cv2.circle(canvas, (int(x), int(y)), 10, (0, 0, 255), -1)
    # plot the actual x,y coordinates
    cv2.circle(canvas, (int(x_actual), int(y_actual)), 10, (0, 255, 0), -1 )
    # show the canvas
    cv2.imshow('Gaze Tracking on Canvas', canvas)
    cv2.waitKey(0)
    # # show the image 
    # cv2.imshow('image', X_test[i])
    # cv2.waitKey(0)
    # # clear the canvas
    
    canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)

cv2.waitKey(0)
cv2.destroyAllWindows()

25/25 [==============================] - 6s 36ms/step


In [11]:
model.save('./models/eye_gaze_v17.h5')